# Main

Essa é a branch "main" do grupo. Nela vai haver um arquivo main.py para simplesmente rodar o bot.

Seria interessante implementar alguma etapa aqui que verifica se há conexão com a internet caso seja preciso.

In [ ]:
from bot.bot_telegram import BotTelegram

if __name__ == "__main__":
    bot = BotTelegram()
    bot.run()

# grupo6-bot

Branch derivada da "main". É aqui onde é feita a criação do bot no Telegram.

In [ ]:
!pip install python-telegram-bot --upgrade

In [ ]:
import os

os.environ["TOKEN_TELEGRAM"] = "8226838069:AAHb_-15PLE_Lh9JuWS3BrYMpxpQlefmEtY"

In [ ]:
import os
import logging
from telegram import Update
from telegram.ext import ApplicationBuilder, ContextTypes, CommandHandler, MessageHandler, filters

# Configurações básicas para o bot funcionar
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Classe main do BotTelegram:
class BotTelegram:

    # Definição do Token. Ele é único para cada Bot (as instruções para rodá-lo no computador de quem irá utilizar o código devem estar no README):
    def __init__(self):
        self._token = os.getenv("TOKEN_TELEGRAM")
        if not self._token:
            raise ValueError("Defina a variável de ambiente TOKEN_TELEGRAM (Vide README.md)")

        self._application = ApplicationBuilder().token(self._token).build()
        self._handlers() # Já define o _handlers aqui para que mudanças neles sejam mais fáceis de serem feitas

    # Definição dos handlers (respostas principais às mensagens do usuário):
    def _handlers(self):
        self._application.add_handler(CommandHandler("start", self.start)) # Inicia o bot
        self._application.add_handler(
            MessageHandler(filters.ALL, self.tratar_mensagem) # Trata a mensagem enviada
        )

    # Mensagem inicial do bot:
    async def start(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        await update.message.reply_text(
            "Olá! Por favor, envie um áudio ou uma imagem, para que eu realize sua transcrição ou reconhecimento."
        )

    # Handler que trata a mensagem e joga para outras classes específicas para cada tarefa, a depender do que foi enviado:
    async def tratar_mensagem(self, update: Update, context: ContextTypes.DEFAULT_TYPE):
        if update.message.text:
            await self.tratar_texto(update, context)
        elif update.message.photo:
            await self.tratar_imagem(update, context)
        elif update.message.voice or update.message.audio:
            await self.tratar_audio(update, context)
        else:
            await update.message.reply_text("Desculpe, a mensagem que você enviou não é de um tipo suportado. Envie apenas imagens ou áudios!")

    # Nessa seção, define a resposta do bot para cada tipo de mensagem enviada:
    async def tratar_texto(self, update, context):
        await update.message.reply_text("Texto recebido com sucesso! No entanto, processar textos está fora do meu escopo.\nPor favor, envie somente áudios ou imagens ;)")

    async def tratar_imagem(self, update, context):
        await update.message.reply_text("Imagem recebida com sucesso! Seu processamento está em andamento...")

    async def tratar_audio(self, update, context):
        await update.message.reply_text("Áudio recebido com sucesso! Seu processamento está em andamento...")

    # Para rodar o código do bot:
    def run(self):
        self._application.run_polling()


# grupo6-img

Essa branch é derivada da "grupo6". É aqui onde fica a subclasse para reconhecimento de imagens.

# grupo6-audio

Essa branch é derivada da "grupo6". É aqui onde fica a subclasse para transcrição de áudios. Coloquei para ele reconhecer tanto áudios enviados pelo Telegram quanto arquivos externos.

In [ ]:
!pip3 install vosk

In [ ]:
# Baixa package para conhecimento de voz em Português-BR
!wget https://alphacephei.com/vosk/models/vosk-model-small-pt-0.3.zip

# Descompacta o arquivo .zip
!unzip vosk-model-small-pt-0.3.zip


In [ ]:
from vosk import Model, KaldiRecognizer
import wave
import tempfile
import json
import subprocess

class BotAudio(BotTelegram):

    async def tratar_audio(self, update, context): # Aqui entra o polimorfismo na classe filha

        # Identifica se é áudio do Telegram ou externo:
        if update.message.voice: # Telegram
          file = await update.message.voice.get_file()
          suffix = ".ogg"

        elif update.message.audio: # Externo
          mime = update.message.audio.mime_type # Pega o tipo do arquivo
          if not mime or not mime.startswith("audio/"): # Se o arquivo não é áudio, rejeita
            await update.message.reply_text("Hmm... Parece que o arquivo que você enviou não é um arquivo de áudio válido. Tente novamente!")
            return

          file = await update.message.audio.get_file()
          suffix = os.path.splitext(update.message.audio.file_name)[1] # Pega formato do arquivo

        else:
          await update.message.reply_text("Muito bem! Envie uma mensagem de voz ou um arquivo de áudio para que eu os transcreva!")
          return

        # Salva temporariamente para o processamento:
        temp_audio = tempfile.NamedTemporaryFile(delete=False, suffix=suffix).name
        await file.download_to_drive(temp_audio)

        # Converte para WAV padrão:
        wav_path = temp_audio.replace(suffix, ".wav")
        subprocess.run([
            "ffmpeg", # O FFMPEG é um "programa" que realiza conversões de arquivos
            "-i", temp_audio, # Define arquivo de entrada
            "-ar", "16000", # Taxa de amostragem do áudio (16kHz)
            "-ac", "1", # Define áudio como mono
            wav_path,
            "-y" # Muda o formato
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        # Abre arquivo WAV convertido:
        wf = wave.open(wav_path, "rb")

        # Estabelece o modelo do Vosk PT-BR (aqui ele chama o mais leve, podemos trocar depois):
        model = Model("vosk-model-small-pt-0.3")
        rec = KaldiRecognizer(model, wf.getframerate())

        # Processa o áudio:
        while True:
            data = wf.readframes(4000) # Processa 4k frames por vez
            if len(data) == 0: # Espera áudio terminar
                break
            rec.AcceptWaveform(data) # Envia para processamento

        texto = json.loads(rec.FinalResult()).get("text", "") # Extrai o texto do Vosk

        # Verifica se o áudio é vazio; se não, transcreve:
        if not texto.strip():
          await update.message.reply_text("Poxa, seu áudio está vazio :(\nQue tal me enviar algo com conteúdo para que eu possa transcrevê-lo?")
        else:
          await update.message.reply_text(f"Muito bem! Aqui está o seu áudio transcrito:\n{texto}")

